# Pandas (3)

## Лабораторная работа №2.2

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [ ]:
import pandas as pd
import numpy as np

recipes = pd.read_csv(
    "data/recipes_sample.csv",
    sep=",",
    header=0
)
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [ ]:
reviews = pd.read_csv(
    "data/reviews_sample.csv",
    sep=",",
    header=0,
    index_col=0
)
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


### Группировки таблиц `pd.DataFrame`

#### 5.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [ ]:
contributed_recipes_count = recipes.groupby("contributor_id")["id"].count()
contributed_recipes_count

contributor_id
1530            5
1533          186
1534           50
1535           40
1538            8
             ... 
2001968497      2
2002059754      1
2002234079      1
2002234259      1
2002247884      1
Name: id, Length: 8404, dtype: int64

In [ ]:
contributed_recipes_count.idxmax(), contributed_recipes_count.max()

(89831, 421)

In [ ]:
contributed_recipes_count.loc[89831]

421

#### 5.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [ ]:
# Посчитайте средний рейтинг к каждому из рецептов.
def avg(group, column):
    return (group[column].sum() / group[column].count())

reviews_by_recipes = reviews.groupby("recipe_id")
avg(reviews_by_recipes, "rating")

# reviews.groupby("recipe_id")["rating"].agg

recipe_id
48        1.000000
55        4.750000
66        4.944444
91        4.750000
94        5.000000
            ...   
536547    5.000000
536610    0.000000
536728    4.000000
536729    4.750000
536747    0.000000
Name: rating, Length: 28100, dtype: float64

In [ ]:
# Для скольких рецептов отсутствуют отзывы?
merged = recipes.merge(
    reviews,
    how="left",
    left_on="id",
    right_on="recipe_id"
)

# Количество отзывов на рецепт
reviews_on_recipe = merged.groupby("id")["recipe_id"].count()

# Рецепты, к которым нет отзывов
recipes_without_reviews = reviews_on_recipe[reviews_on_recipe == 0]
recipes_without_reviews.head()

id
1144    0
2691    0
2759    0
2994    0
3145    0
Name: recipe_id, dtype: int64

In [ ]:
recipes_without_reviews.count()

1900

#### 5.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [ ]:
# Конвертация в datetime
recipes["submitted"] = pd.to_datetime(recipes["submitted"], format="%Y-%m-%d")

# Добавление столбца year
years = recipes["submitted"].map(lambda x: x.year)
years.name = "year"
recipes_by_year = pd.concat([recipes, years], axis=1)

# Группировка по году создания
recipes_by_year.groupby("year")["id"].count().head()

year
1999     275
2000     104
2001     589
2002    2644
2003    2334
Name: id, dtype: int64

### Объединение таблиц `pd.DataFrame`

#### 5.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
merged = recipes.merge(
    reviews,
    how="inner",
    left_on="id",
    right_on="recipe_id"
)[["id", "name", "user_id", "rating"]]
merged.head()

,id,name,user_id,rating
0,44123,george s at the cove black bean soup,743566,5
1,44123,george s at the cove black bean soup,76503,5
2,44123,george s at the cove black bean soup,34206,5
3,67664,healthy for them yogurt popsicles,494084,5
4,67664,healthy for them yogurt popsicles,303445,5


In [ ]:
# выбрав рецепт, не имеющий отзывов
reviews[reviews["recipe_id"] == 1144]

,user_id,recipe_id,date,rating,review


In [ ]:
# попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.
testname = recipes[recipes["id"] == 1144]
testname

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
25816,steak tomato basil pasta,1144,0,1533,1999-08-09,NaN,NaN,11.0


In [ ]:
merged[merged["name"] == "steak tomate basil pasta"]

,id,name,user_id,rating


In [ ]:
merged[merged["name"] == "george s at the cove  black bean soup"]

,id,name,user_id,rating
0,44123,george s at the cove black bean soup,743566,5
1,44123,george s at the cove black bean soup,76503,5
2,44123,george s at the cove black bean soup,34206,5


#### 5.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
merged = recipes.merge(
    reviews,
    how="left",
    left_on="id",
    right_on="recipe_id"
)[["id", "name", "user_id"]]

reviews_on_recipe = merged.groupby("id")["user_id"].count()
reviews_on_recipe.name = "review_count"
reviews_on_recipe.head()

id
48     2
55     4
66    18
91     4
94     4
Name: review_count, dtype: int64

In [ ]:
df = recipes[["id", "name"]].merge(
    reviews_on_recipe,
    how="inner",
    on="id"
)

df.rename(columns={"id":"recipe_id"})

,recipe_id,name,review_count
0,44123,george s at the cove black bean soup,3
1,67664,healthy for them yogurt popsicles,8
2,38798,i can t believe it s spinach,3
3,35173,italian gut busters,1
4,84797,love is in the air beef fondue sauces,8
...,...,...,...
29995,267661,zurie s holey rustic olive and cheddar bread,4
29996,386977,zwetschgenkuchen bavarian plum cake,2
29997,103312,zwiebelkuchen southwest german onion cake,6
29998,486161,zydeco soup,6


#### 5.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [ ]:
# recipes_by_year - из задания 5.3
year_rating = recipes_by_year.merge(
    reviews,
    how="inner",
    left_on="id",
    right_on="recipe_id"
)[["year", "rating"]]Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

group = year_rating.groupby("year")

# avg - из задания 5.2
avg_rating_by_year = avg(group, "rating").head()
avg_rating_by_year

year
1999    4.274895
2000    4.284585
2001    4.393945
2002    4.404645
2003    4.439152
Name: rating, dtype: float64

In [ ]:
# idxmin - Return index of first occurrence of minimum over requested axis.
avg_rating_by_year.idxmin(), avg_rating_by_year.min()

(1999, 4.274894810659187)

### Сохранение таблиц `pd.DataFrame`

#### 6.1 Отсортируйте результат выполнения задания 6.1 в порядке убывания величины столбца `id` и сохраните результаты в csv файл. 

In [ ]:
#3.1
# Добавление столбца с длиной описания рецепта
recipes['description_length'] = recipes['description'].fillna('').apply(len)

# Сортировка таблицы по убыванию значения столбца "name_word_count"
recipes_sorted1 = recipes.sort_values('name_word_count', ascending=False)

# Сохранение отсортированной таблицы в CSV файл
recipes_sorted1.to_csv('recipes_sorted_by_name_word_count1.csv', index=False)

#3.2
# Изменение регистра каждого слова в столбце "name" на заглавный
recipes['name'] = recipes['name'].apply(lambda x: x.title())

# Сортировка таблицы по убыванию значения столбца "name_word_count"
recipes_sorted2 = recipes.sort_values('name_word_count', ascending=False)

# Сохранение отсортированной таблицы в новый CSV файл
recipes_sorted2.to_csv('recipes_sorted_by_name_word_count2.csv', index=False)

#3.3
# Добавление столбца с количеством слов в названии рецепта
recipes['name_word_count'] = recipes['name'].apply(lambda x: len(x.split()))

# Сортировка таблицы по убыванию значения столбца "name_word_count"
recipes_sorted3 = recipes.sort_values('name_word_count', ascending=False)
recipes_sorted3.to_csv('recipes_sorted_by_name_word_count3.csv', index=False)

#### 6.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 5.1 и 5.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 5.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 5.2.

In [ ]:
with pd.ExcelWriter('output.xlsx') as writer:  
    merged.to_excel(writer, sheet_name='Рецепты с оценками')
    df.to_excel(writer, sheet_name='Количество отзывов по рецептам')